In [120]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import RandomOverSampler
from sklearn.utils.class_weight import compute_class_weight

In [113]:
df1=pd.read_csv('/content/dataset_diabetes_1.csv')
df2=pd.read_csv('/content/dataset_diabetes_2.csv')
df= pd.concat([df1, df2], ignore_index=True)
X = df[['Age', 'Blood Glucose Levels', 'Blood Pressure', 'Weight Gain During Pregnancy',
        'Waist Circumference', 'BMI', 'Insulin Levels', 'Cholesterol Levels',
        'Digestive Enzyme Levels', 'Pulmonary Function']]
y = df['Target']
# Split data into training and temporary sets (80% train, 20% temp)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42) #random_state for reproducibility

# Split the temporary set into validation and testing sets (50% validation, 50% test)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Now you have:
print(len(X_train), len(y_train))
print(len(X_val), len(y_val))
print(len(X_test), len(y_test))

96355 96355
12044 12044
12045 12045


In [114]:
X = X.fillna(X.median())
print(X.shape)
print(X.isnull().sum().sum())  # Harus 0

(120444, 10)
0


In [121]:
scaler = StandardScaler()
X = scaler.fit_transform(X)
encoder = LabelEncoder()
y = encoder.fit_transform(y)

In [122]:
# Split data into training and temporary sets (80% train, 20% temp)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)

# Split the temporary set into validation and testing sets (50% validation, 50% test)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Validasi ukuran data
print("Train size:", len(X_train), len(y_train))
print("Validation size:", len(X_val), len(y_val))
print("Test size:", len(X_test), len(y_test))


Train size: 96355 96355
Validation size: 12044 12044
Test size: 12045 12045


In [124]:
# Lakukan oversampling pada data latih
ros = RandomOverSampler(random_state=42)
X_train_balanced, y_train_balanced = ros.fit_resample(X_train, y_train)
# Hitung bobot kelas
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(enumerate(class_weights))

print("Class weights:", class_weights)


Class weights: {0: 5.2820414428242515, 1: 5.415027537372148, 2: 5.462301587301587, 3: 5.3518662519440126, 4: 5.650656814449918, 5: 5.506, 6: 5.45796986518636, 7: 5.364380358534684, 8: 5.286098310291859, 9: 5.475338106603023, 10: 5.44501582278481, 11: 5.586444805194805, 12: 5.265876052027544, 13: 0.08618098946920275}


In [125]:
# Definisikan model
zaidan = tf.keras.Sequential([
    tf.keras.layers.Reshape((X_train.shape[1], 1), input_shape=(X_train.shape[1],)),  # Reshape data input
    tf.keras.layers.Conv1D(32, 3, activation='relu'),  # Layer Conv1D pertama
    tf.keras.layers.MaxPooling1D(2),                  # MaxPooling
    tf.keras.layers.Conv1D(64, 3, activation='relu'), # Layer Conv1D kedua
    tf.keras.layers.MaxPooling1D(2),                  # MaxPooling
    tf.keras.layers.Flatten(),                        # Flatten untuk data 1D menjadi 2D
    tf.keras.layers.Dense(128, activation='relu'),    # Dense layer dengan 128 neuron
    tf.keras.layers.Dropout(0.2),                     # Dropout untuk mengurangi overfitting
    tf.keras.layers.Dense(len(np.unique(y_train)), activation='softmax')  # Output layer sesuai jumlah kelas
])

# Kompilasi model
zaidan.compile(
    loss='sparse_categorical_crossentropy',  # Karena y_train berbentuk integer skalar
    optimizer='adam',
    metrics=['accuracy', 'mae', 'mse']
)

# Lihat ringkasan model
zaidan.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ reshape_5 (Reshape)                  │ (None, 10, 1)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_10 (Conv1D)                   │ (None, 8, 32)               │             128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_10 (MaxPooling1D)      │ (None, 4, 32)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_11 (Conv1D)                   │ (None, 2, 64)               │           6,208 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_11 (MaxPooling1D)      │ (None, 1, 64)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_5 (Flatten)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 14)                  │           1,806 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 16,462 (64.30 KB)

 Trainable params: 16,462 (64.30 KB)

 Non-trainable params: 0 (0.00 B)

In [127]:
zaidan.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']  # Gunakan hanya akurasi
)

In [129]:
history = zaidan.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=128,
    validation_data=(X_val, y_val),
    verbose=1
)


Epoch 1/100
753/753 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.8899 - loss: 0.6972 - val_accuracy: 0.9513 - val_loss: 0.1235
Epoch 2/100
753/753 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9477 - loss: 0.1285 - val_accuracy: 0.9582 - val_loss: 0.1041
Epoch 3/100
753/753 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.9559 - loss: 0.1106 - val_accuracy: 0.9647 - val_loss: 0.0950
Epoch 4/100
753/753 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9600 - loss: 0.0993 - val_accuracy: 0.9630 - val_loss: 0.0924
Epoch 5/100
753/753 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9605 - loss: 0.0958 - val_accuracy: 0.9621 - val_loss: 0.0957
Epoch 6/100
753/753 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.9620 - loss: 0.0940 - val_accuracy: 0.9662 - val_loss: 0.0873
Epoch 7/100
753/753 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.9638 - loss: 0.0889 - val_accuracy: 0.9678 - val_loss: 0.0837
Epoch 8/100
753/753 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.9656 - loss: 0.0853 - val_accu

In [130]:
# Evaluasi model
loss, accuracy = zaidan.evaluate(X_test, y_test, verbose=0)

print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

# Prediksi pada data uji
y_pred = zaidan.predict(X_test)
y_pred_classes = tf.argmax(y_pred, axis=1).numpy()  # Konversi ke numpy array
y_pred_probs = np.max(y_pred, axis=1)

# Contoh prediksi
print("\nExample Predictions:")
for i in range(20):
    print(f"Sample {i+1}: Predicted Class - {y_pred_classes[i]}, Actual Class - {y_test[i]}, Probability - {y_pred_probs[i]:.4f}")



Test Loss: 0.0921
Test Accuracy: 0.9698
377/377 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step

Example Predictions:
Sample 1: Predicted Class - 13, Actual Class - 13, Probability - 1.0000
Sample 2: Predicted Class - 13, Actual Class - 13, Probability - 1.0000
Sample 3: Predicted Class - 13, Actual Class - 13, Probability - 1.0000
Sample 4: Predicted Class - 13, Actual Class - 13, Probability - 1.0000
Sample 5: Predicted Class - 13, Actual Class - 13, Probability - 1.0000
Sample 6: Predicted Class - 13, Actual Class - 13, Probability - 1.0000
Sample 7: Predicted Class - 13, Actual Class - 13, Probability - 1.0000
Sample 8: Predicted Class - 13, Actual Class - 13, Probability - 1.0000
Sample 9: Predicted Class - 13, Actual Class - 13, Probability - 1.0000
Sample 10: Predicted Class - 9, Actual Class - 9, Probability - 1.0000
Sample 11: Predicted Class - 13, Actual Class - 13, Probability - 1.0000
Sample 12: Predicted Class - 13, Actual Class - 13, Probability - 1.0000
Sample 13: Predicted Class - 13,

In [131]:
from tensorflow.keras.models import save_model
from google.colab import files

# Simpan model ke penyimpanan sementara Colab
model_path = "zaidan_model.keras"
zaidan.save(model_path)

# Download file model
files.download(model_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>